In [1]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize

import pydicom
from pydicom.data import get_testdata_files

import os
from tensorboardX import SummaryWriter

from datasets import Brains
from modules import Conv2D1, ConvTranspose2D1
from models import UNet3D

import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

In [4]:
class UNet3D_AE(UNet3D):
    def forward(self, x):
        x_in = self.inconv(x)
        #         print(x_in.shape)
        x1 = self.down1(x_in)
        #         print(x1.shape)

        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)
        print(x4.shape)
        print(x3.shape)
        x = self.up1(x4, x3)
        x = self.up2(x, x2)
        x = self.up3(x, x1)
        x = self.up4(x, x_in)

        return self.outconv(x)

In [11]:
!nvidia-smi

Sat Nov 17 16:45:44 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   40C    P0    55W / 300W |   7046MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:1C:00.0 Off |                    0 |
| N/A   52C    P0   151W / 300W |   4263MiB / 16160MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [10]:
data = Brains(crop_size=(64,64))
batch_size = 2
datait = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=20)

In [12]:
unet_ae = UNet3D_AE(input_nc=1, output_nc=2)

In [14]:
unet_ae(data[0][0])

RuntimeError: Expected 5-dimensional input for 5-dimensional weight [48, 1, 3, 3, 3], but got input of size [1, 24, 64, 64] instead